In [16]:
import spacy

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [18]:
data=pd.read_excel("List_Songs-HIPPOP.xlsx")

In [19]:
data2=pd.concat([data, pd.get_dummies(data['Genre'])], axis=1)

In [20]:
#data2

In [21]:
#Husk at vi her har importeret Spacy!!!
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [22]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [23]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [24]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [25]:
#######"#DET ER HERFRA DATA SPECIFICERES!########

In [26]:
data2.head(2)

,Lyrics,Title,Year,Genre,Hip Hop,Pop
1,Uh uh uh\nI just gotta bring it to they attent...,#1,2002,Hip Hop,1,0
2,\n\nChorus (2x):(Mannie Fresh)\nAll the nigga...,#1,2002,Hip Hop,1,0


In [39]:
from sklearn.model_selection import train_test_split

X = data2['Lyrics'] # the features we want to analyze
ylabels = data2['Hip Hop'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [40]:
#log = LogisticRegression(penalty='l1', solver='liblinear')
#log.fit(X, y)

In [41]:
from sklearn.linear_model import LogisticRegressionCV

In [42]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
#Er det her vi skal specificere niveau af regulization?
classifier = LogisticRegressionCV(cv=50)

#HER HAR VI DE TO FORSKELLIGE GENERALIZATIONS# l1=Lasso, l2=Ridge

#(penalty='l1', solver='liblinear')
#(penalty='l2')

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x1a169af0f0>), ('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ng...    random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0))])

In [44]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7923076923076923
Logistic Regression Precision: 0.8461538461538461
Logistic Regression Recall: 0.6984126984126984
